# Photon quality check

In [1]:
%pip install --quiet "sliderule>=4.6.2"

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import scipy
import skimage
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from tqdm import tqdm
import shapely
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from sliderule import sliderule, icesat2, earthdata
import ipyleaflet
from datetime import datetime, timedelta
from shapely import Polygon

In [2]:
url = "slideruleearth.io"
sliderule.icesat2.init(url, verbose=True)
asset = "icesat2"

In [31]:
# search CMR for matching ATL03 in the bounding box

parms = {
    # processing parameters
    "srt": icesat2.SRT_LAND,
    "len": 20,
    "res": 20,
    # classification and checks
    # still return photon segments that fail checks
    "pass_invalid": True, 
    # all photons
    "cnf": -2, 
    # all photons
    "yapc": dict(knn=0, win_h=6, win_x=11, min_ph=4, score=0), 
}

# ICESat-2 data release
release = '006'

# region of interest
poly = [{'lat': 50.6893, 'lon': -2.9042},
        {'lat': 50.6893, 'lon': -2.7463},
        {'lat': 50.2320, 'lon': -2.9015},
        {'lat': 50.2320, 'lon': -2.7463}]

# time bounds for query
time_start = '2019-12-14'
time_end = '2020-01-30'

# find granules for the region of interest
granules_list = earthdata.cmr(short_name='ATL03', polygon=poly, time_start=time_start, time_end=time_end, version=release)


parms["poly"] = poly
gdf = icesat2.atl03sp(parms, resources=granules_list)

In [32]:
# granules list
granules_list

['ATL03_20191225070154_13650506_006_01.h5',
 'ATL03_20200123053755_04200606_006_01.h5']

In [33]:
gdf

,track,solar_elevation,rgt,pair,region,segment_dist,segment_id,sc_orient,cycle,background_rate,...,atl03_cnf,landcover,height,relief,atl08_class,quality_ph,snowcover,y_atc,geometry,spot
time,,,,,,,,,,,,,,,,,,,,,
2019-12-25 07:04:12.760312064,1,-9.930334,1365,0,6,1.442550e+07,720229,1,5,16604.631784,...,4,255,50.851685,0.0,4,0,255,12386.559570,POINT (-2.74793 50.68938),6
2019-12-25 07:04:12.760312064,1,-9.930334,1365,0,6,1.442550e+07,720229,1,5,16604.631784,...,4,255,51.081795,0.0,4,0,255,12386.553711,POINT (-2.74793 50.68938),6
2019-12-25 07:04:12.760412160,1,-9.930334,1365,0,6,1.442550e+07,720229,1,5,16604.631784,...,4,255,51.053402,0.0,4,0,255,12386.557617,POINT (-2.74793 50.68937),6
2019-12-25 07:04:12.760512000,1,-9.930334,1365,0,6,1.442550e+07,720229,1,5,16604.631784,...,4,255,50.895374,0.0,4,0,255,12386.565430,POINT (-2.74793 50.68937),6
2019-12-25 07:04:12.760512000,1,-9.930334,1365,0,6,1.442550e+07,720229,1,5,16604.631784,...,4,255,50.995960,0.0,4,0,255,12386.562500,POINT (-2.74793 50.68937),6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-25 07:04:16.588312064,2,-9.874628,1365,1,6,1.445036e+07,721470,1,5,16753.880422,...,4,255,50.687702,0.0,4,0,255,9125.265625,POINT (-2.82831 50.46966),3
2019-12-25 07:04:16.588312064,2,-9.874628,1365,1,6,1.445036e+07,721470,1,5,16753.880422,...,4,255,46.147923,0.0,4,0,255,9125.362305,POINT (-2.82831 50.46966),3
2019-12-25 07:04:16.588312064,2,-9.874628,1365,1,6,1.445036e+07,721470,1,5,16753.880422,...,4,255,50.697826,0.0,4,0,255,9125.265625,POINT (-2.82831 50.46966),3


## Test 1
#### Noise percentage

In [40]:
# Extract unique values from the DataFrame
rgt_values = gdf['rgt'].unique()
track_values = gdf['track'].unique()
pair_values = gdf['pair'].unique()

# Loop through all combinations of rgt, track, and pair
for rgt in rgt_values:
    for track in track_values:
        for pair in pair_values:
            mask = (gdf['rgt'] == rgt) & (gdf['track'] == track) & (gdf['pair'] == pair)
            total_count = mask.sum()
            noise_count = (mask & (gdf['atl03_cnf'] == 0)).sum()
            noise_percentage = (noise_count / total_count) * 100 if total_count > 0 else 0

            print(f"rgt: {rgt}, track: {track}, pair: {pair} - Noise percentage: {noise_percentage:.2f}%")


rgt: 1365, track: 1, pair: 0 - Noise percentage: 0.81%
rgt: 1365, track: 1, pair: 1 - Noise percentage: 0.90%
rgt: 1365, track: 2, pair: 0 - Noise percentage: 1.21%
rgt: 1365, track: 2, pair: 1 - Noise percentage: 0.86%
rgt: 1365, track: 3, pair: 0 - Noise percentage: 1.26%
rgt: 1365, track: 3, pair: 1 - Noise percentage: 0.75%


## Test 2
#### Confidence

In [41]:
# Initialize a list to store results
valid_granules = []

# Define the threshold percentage
threshold_percentage = 70  # 70%

# Loop through all combinations of rgt, track, and pair
for rgt in rgt_values:
    for track in track_values:
        for pair in pair_values:
            # Boolean mask for filtering
            mask = (gdf['rgt'] == rgt) & (gdf['track'] == track) & (gdf['pair'] == pair)
            
            # Total count of photons
            total_count = mask.sum()
            
            if total_count == 0:
                continue  # Skip if no photons are present
            
            # Count the number of photons with YAPC > 150
            high_yapc_count = (mask & (gdf['yapc_score'] > 150)).sum()
            
            # Calculate the percentage of high YAPC photons
            high_yapc_percentage = (high_yapc_count / total_count) * 100
            
            # Check if the percentage meets the threshold
            if high_yapc_percentage >= threshold_percentage:
                valid_granules.append({
                    'rgt': rgt,
                    'track': track,
                    'pair': pair,
                    'high_yapc_percentage': high_yapc_percentage
                })

# Print the results
for granule in valid_granules:
    print(f"rgt: {granule['rgt']}, track: {granule['track']}, pair: {granule['pair']} - Percentage of photons with YAPC > 150: {granule['high_yapc_percentage']:.2f}%")


rgt: 1365, track: 1, pair: 0 - Percentage of photons with YAPC > 150: 93.53%
rgt: 1365, track: 1, pair: 1 - Percentage of photons with YAPC > 150: 93.84%
rgt: 1365, track: 2, pair: 0 - Percentage of photons with YAPC > 150: 81.09%
rgt: 1365, track: 2, pair: 1 - Percentage of photons with YAPC > 150: 87.31%
rgt: 1365, track: 3, pair: 0 - Percentage of photons with YAPC > 150: 83.52%
rgt: 1365, track: 3, pair: 1 - Percentage of photons with YAPC > 150: 88.13%
